<a href="https://colab.research.google.com/github/heyibad/genai-projects/blob/main/rag_pinecone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip  install -qU langchain-google-genai langchain-pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.2/412.2 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 4.2 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
from pinecone import Pinecone, ServerlessSpec

PINECONE_API_KEY = userdata.get('PINECONE_API_KEY')

pc = Pinecone(api_key=PINECONE_API_KEY)


In [ ]:

index_name = "langchain-rag-01"

# pc.create_index(
#         name=index_name,
#         dimension=768, # for google model dimentions are 768
#         metric="cosine",
#         spec=ServerlessSpec(cloud="aws", region="us-east-1"),
#     )

index = pc.Index(index_name)

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

GOOGLE_API = userdata.get("GEMINI_API_KEY")

embeddings = GoogleGenerativeAIEmbeddings(google_api_key=GOOGLE_API, model="models/embedding-001")

In [ ]:
vector = embeddings.embed_query("Hello world")
vector[0:5]

[0.04703257977962494,
 -0.04019005596637726,
 -0.02902696467936039,
 -0.026809632778167725,
 0.01892058178782463]

In [ ]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(
    # a index specific project where it stores, & Embeddings model
    index=index,
    embedding=embeddings
)

In [ ]:
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]


In [ ]:
uuids = [str(uuid4()) for _ in range(len(documents))]
#             random_ids --                     10

vector_store.add_documents(documents=documents, ids=uuids)

['355f280c-6314-4840-8a49-91c9cf0cebb2',
 '62577437-f0ab-486a-8b80-1244d325869c',
 '866de3d0-83dc-4402-812a-2a16f5110553',
 '5d1f39e9-7e4d-4dc1-b4e2-5e43540c7898',
 '51a399be-a907-4f09-8376-73f6ff9ff597',
 'fd5fcb7c-ae52-403c-94f5-d060c78662b1',
 '2097d2fb-b8c7-4771-8c65-9de3afdc12ab',
 '9221c0ef-f20d-4cb0-b047-69f8777ec67f',
 '06c69fca-8314-4abc-9ad0-f317c087f511',
 'f3c8c43b-0234-477f-8c52-4b1c7e6898b7']

In [ ]:
result = vector_store.similarity_search("Langchain works",k=2,filter={"source":"tweet"})

for res in result:
  print(f"the results are: {res.page_content}")

the results are: Building an exciting new project with LangChain - come check it out!
the results are: LangGraph is the best framework for building stateful, agentic applications!


In [ ]:
results = vector_store.similarity_search_with_score(
    "Will it be hot tomorrow?", k=1, filter={"source": "news"}
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.668031] The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news'}]


In [ ]:
from typing import Dict

def vec_search(query:str,max_result:int=1)->None:
      result = vector_store.similarity_search(query,k=max_result)

      for res in result:
          print(f"the result is: {res.page_content}")

In [ ]:
vec_search("What is Langchain?")


the result is: Building an exciting new project with LangChain - come check it out!
